In [131]:
import random
import numpy as np 
import pandas as pd
import operator
from scipy.sparse import coo_matrix
from numpy.linalg import norm
from sklearn.metrics import mean_squared_error

In [132]:
# from google.colab import drive
# drive.mount('/content/drive')

In [133]:
ls

drive/  sample_data/


In [134]:
import os, sys 
from google.colab import drive 

### 해당 코드 실행 시 colab에서 실행중인 폴더의 /content/drive/My Drive가 구글 드라이브에 연결됨

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [135]:
#### 영화 데이터셋 형태 확인 ####
movies = pd.read_csv("drive/MyDrive/data/others/movies.csv")

print("### Movie Dataset Format ###", end = '\n\n')
movies.head()

### Movie Dataset Format ###



,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [136]:
################### 가상 유저 추가해주기 위한 작업 데이터 전처리 - 장르별 영화 리스트를 뽑는다 #################

# 이 셀을 실행하는 이유는 후에 특정 장르를 좋아하는 가상의 유저를 만들어서 추천의 결과를 보기 위함입니다.

movie_dict = dict()                 # {movie_id : (movie_title, movie_genre)}
musical_list = list()               # musical 장르의 영화들
horror_list = list()                # horror 장르의 영화들
documentary_list = list()           # documentary 장르의 영화들
comedy_list = list()                # comedy 장르의 영화들
animation_list = list()

for (movie_id, movie_title, movie_genre) in movies.itertuples(index=False):
    movie_dict[movie_id] = (movie_title, movie_genre)
    if 'Musical' in movie_genre:
        musical_list.append(movie_id)
    if 'Horror' in movie_genre:
        horror_list.append(movie_id)
    if 'Documentary' in movie_genre:
        documentary_list.append(movie_id)
    if 'Comedy' in movie_genre:
        comedy_list.append(movie_id)
    if 'Animation' in movie_genre:
        animation_list.append(movie_id)

In [137]:
ratings = pd.read_csv("drive/MyDrive/data/others/ratings.csv")

# 평점 데이터셋 형태 확인
print("### Rating Dataset Format ###", end='\n\n')
print(ratings.head(), end='\n\n\n')
ratings.drop(['timestamp'], axis=1, inplace=True)
print(ratings.head())

### Rating Dataset Format ###

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


   userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0
4       1       50     5.0


In [138]:
'''
    유저가 평균적으로 영화를 130개정도 본 것으로 나타나서 일반적으로 추가하는 사람들도 그정도로 설정
    그리고, 영화에 대한 평점을 줄 때는 영화 순서를 섞고, 무조건 5점을 주는 것이 아니라 4, 5점 중 랜덤으로 설정
    낮은 점수를 줄 때도 마찬가지.
'''
# uid==800을 가지는 유저를 추가한다. 이 유저는 뮤지컬의 광팬이다.

random.shuffle(musical_list);
random.shuffle(horror_list);
random.shuffle(documentary_list);
random.shuffle(comedy_list);

print(ratings)
new_uid = 800
rows = list()
for movie_id in musical_list[:100]:
    rows.append([new_uid, movie_id, random.randint(4,5)])
for movie_id in horror_list[:50]:
    rows.append([new_uid, movie_id, random.randint(1,2)])
for movie_id in documentary_list[:20]:
    rows.append([new_uid, movie_id, random.randint(2,3)])
for row in rows:
    ratings = ratings.append(pd.Series(row, index=ratings.columns), ignore_index=True)
print(ratings)


        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
100831     610   166534     4.0
100832     610   168248     5.0
100833     610   168250     5.0
100834     610   168252     5.0
100835     610   170875     3.0

[100836 rows x 3 columns]
        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
101001     800    61236     3.0
101002     800     5669     3.0
101003     800     1189     3.0
101004     800   178323     3.0
101005     800     2330     3.0

[101006 rows x 3 columns]


In [139]:
# uid==850을 가지는 유저를 추가한다. 이 유저는 다큐 영화의 광팬이다.

random.shuffle(musical_list);
random.shuffle(horror_list);
random.shuffle(documentary_list);
random.shuffle(comedy_list);
random.shuffle(animation_list);


print(ratings)
new_uid = 850
rows = list()
for movie_id in documentary_list[:100]:
    rows.append([new_uid, movie_id, random.randint(4,5)])
for movie_id in horror_list[:50]:
    rows.append([new_uid, movie_id, random.randint(1,2)])
for movie_id in animation_list[:10]:
    rows.append([new_uid, movie_id, random.randint(1,2)])

for row in rows:
    ratings = ratings.append(pd.Series(row, index=ratings.columns), ignore_index=True)
print(ratings)

        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
101001     800    61236     3.0
101002     800     5669     3.0
101003     800     1189     3.0
101004     800   178323     3.0
101005     800     2330     3.0

[101006 rows x 3 columns]
        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
101161     850    84414     1.0
101162     850    98799     2.0
101163     850   174141     2.0
101164     850   163809     2.0
101165     850   136016     1.0

[101166 rows x 3 columns]


In [140]:
# uid==900을 가지는 유저를 추가한다. 이 유저는 호러의 광팬이다.

random.shuffle(musical_list);
random.shuffle(horror_list);
random.shuffle(documentary_list);
random.shuffle(comedy_list);
random.shuffle(animation_list);

new_uid = 900
rows = list()
for movie_id in horror_list[:120]:
    rows.append([new_uid, movie_id, random.randint(4,5)])
for movie_id in documentary_list[:10]:
    rows.append([new_uid, movie_id, random.randint(1,2)])
for movie_id in comedy_list[:30]:
    rows.append([new_uid, movie_id, random.randint(1,2)])
for movie_id in animation_list[:20]:
    rows.append([new_uid, movie_id, random.randint(1,2)])
for row in rows:
    ratings = ratings.append(pd.Series(row, index=ratings.columns), ignore_index=True)
print(ratings)

        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
101341     900    97913     2.0
101342     900     2123     2.0
101343     900   136297     2.0
101344     900    33090     1.0
101345     900     5041     2.0

[101346 rows x 3 columns]


In [141]:
# uid==950을 가지는 유저를 추가한다. 이 유저는 애니메이션의 광팬이다.

random.shuffle(musical_list);
random.shuffle(horror_list);
random.shuffle(documentary_list);
random.shuffle(comedy_list);
random.shuffle(animation_list);

new_uid = 950
rows = list()
for movie_id in horror_list[:20]:
    rows.append([new_uid, movie_id, random.randint(1,2)])
for movie_id in documentary_list[:10]:
    rows.append([new_uid, movie_id, random.randint(1,3)])
for movie_id in comedy_list[:30]:
    rows.append([new_uid, movie_id, random.randint(2,4)])
for movie_id in animation_list[:150]:
    rows.append([new_uid, movie_id, random.randint(3,5)])
for row in rows:
    ratings = ratings.append(pd.Series(row, index=ratings.columns), ignore_index=True)
print(ratings)

        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
101551     950    36708     5.0
101552     950    81847     5.0
101553     950   118894     3.0
101554     950     6857     5.0
101555     950   101142     4.0

[101556 rows x 3 columns]


In [142]:
########################################################################
######### Mapping user id, movie id to user index, movie index #########
########################################################################

uid_2_idx = dict()     # user id --> user idx
mid_2_idx = dict()    # movie id --> movie idx

u_idx = 0
m_idx = 0
for user_id, movie_id, r in ratings.itertuples(index=False):
    if user_id not in uid_2_idx:
        uid_2_idx[user_id] = u_idx
        u_idx = u_idx+1

    if movie_id not in mid_2_idx:
        mid_2_idx[movie_id] = m_idx
        m_idx = m_idx+1

num_user = len(uid_2_idx)
num_movie = len(mid_2_idx)
print(num_user) # 유저수
print(num_movie) # 영화수

uidx_2_id = {v:k for (k,v) in uid_2_idx.items()}    # user idx --> user id
midx_2_id = {v:k for (k,v) in mid_2_idx.items()}    # movie idx --> movie id


614
9724


In [143]:
# rating matrix 초기화
rating_matrix = np.zeros((num_user, num_movie)) # num_user x num_movie를 크기로 가지는 numpy 행렬 선언, 각 요소는 유저가 영화에 매긴 rating을 나타냄.

for user_id, movie_id, r in ratings.itertuples(index=False):
    u_idx = uid_2_idx[user_id]
    m_idx = mid_2_idx[movie_id]
    rating_matrix[u_idx, m_idx] = r

rating_matrix

array([[4., 4., 4., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [144]:
######################################################################################################################################
# Training Set과 Test Set을 분리해 주는 함수
# kind='user'인 경우 같은 user 내의 rating을, kind = 'item'인 경우 같은 item(movie) 내의 rating을 train set과 test set으로 분리해준다.
######################################################################################################################################
def train_test_split(ratings):
    test = np.zeros_like(ratings)
    train = ratings.copy()
    for x in range(ratings.shape[0]):
        nonzero_idx = ratings[x, :].nonzero()[0]
        test_ratings = np.random.choice(nonzero_idx, 
                                        size=int(len(nonzero_idx)/5),  
                                        replace=False)
        train[x, test_ratings] = 0.
        test[x, test_ratings] = ratings[x, test_ratings]
        
    assert(np.all((train * test) == 0))     # train set과 test set이 완전히 분리되었는지 확인

    return train, test

######################################################################################
###################### 유저별 평균을 기존 rating에서 빼준다 ##########################
######################################################################################
####### *****rating이 "있는" 값들의 평균만을 rating이 있는 곳에서만 빼준다############
def subtract_mean_ratings(ratings):
    mean_subtracted_ratings = np.zeros_like(ratings)
    avg_ratings = np.zeros(ratings.shape[0])
    for i in range(ratings.shape[0]):
        nonzero_idx = ratings[i].nonzero()[0]                       # rating 값이 존재하는(0이 아닌) index 추출
        sum_ratings = np.sum(ratings[i])          
        num_nonzero = len(nonzero_idx)
        avg_rating = sum_ratings / num_nonzero                      # rating 값들의 평균값 계산
        if num_nonzero == 0:                  
            avg_rating = 0
        avg_ratings[i] = avg_rating
        mean_subtracted_ratings[i, nonzero_idx] = ratings[i, nonzero_idx] - avg_rating 
                                                                    # 원 rating matrix에서 평균 값을 빼줌
    return mean_subtracted_ratings, avg_ratings


In [145]:
train_ratings, test_ratings = train_test_split(rating_matrix)
mean_subtracted_ratings, avg_ratings = subtract_mean_ratings(train_ratings)

In [146]:
def calculate_rmse(R, U, V, lambda_u, lambda_v):
    error = 0
    for u, i, r_ui in zip(R.row, R.col, R.data):
        error += (r_ui - np.dot(U[u] , V.T[i]))**2
    for u in range(U.shape[0]):
        error += lambda_u * (norm(U[u])**2) # L2 Normalization
    for i in range(V.shape[1]):
        error += lambda_v * (norm(V.T[i])**2) # L2 Normalization
    rmse = np.sqrt(error/len(R.data))
    return rmse

def SGD(U, V, R, lr, lambda_u, lambda_v):
    for u, i, r_ui in R:
        e = r_ui - np.dot(U[u],V.T[i])
        grad_u = 2 * e * -V.T[i] + 2 * lambda_u * U[u]
        grad_v = 2 * e * -U[u] + 2 * lambda_v * V.T[i]
        U[u] = U[u] - lr * grad_u
        V.T[i] = V.T[i] - lr * grad_v
        
    return U, V


def train(ratings, dim=10, max_epoch=50, lambda_u=0.1, lambda_v=0.1, lr=0.01):
    #lambda_u : regularization coefficient of U matrix, lambda_v : regularizatoin coeff of V matrix,
    # lr : learning rate
    num_u, num_i = ratings.shape

    U = np.random.rand(num_u, dim)
    V = np.random.rand(dim, num_i)
    R = coo_matrix(ratings)
    rmse = calculate_rmse(R, U, V, lambda_u, lambda_v)
    print("Initial RMSE: " + str(rmse))

    for epoch in range(max_epoch):
        if epoch != 0 and epoch%10==0:
            lr /= 2
        rating_data = list(zip(R.row, R.col, R.data))
        random.shuffle(rating_data)
        U, V = SGD(U, V, rating_data, lr, lambda_u, lambda_v)
        rmse = calculate_rmse(R, U, V, lambda_u, lambda_v)        
        print('Epoch: {:5}, RMSE: {:15}, Learning Rate:{}'.format(epoch, rmse, lr))
    
    return U, V 

def predict(U, V, user_id=None):
    # user_id가 주어지지 않은 경우엔 전체 예측된 평점 행렬을 return,
    # 주어진 경우엔 해당 유저에 대한 예측 평점 행렬을 return한다.

    if user_id is None:
        return np.dot(U, V)
    else:
        user_idx = uid_2_idx[user_id]
        return np.dot(U[user_idx], V )


#######################################################################################
##########user id = uid인 유저에게 유저가 본 영화를 제외하고 N개를 추천해준다######
#######################################################################################
def recommend_for_uid(uid, ori_rating_matrix, U, V, top_N):

    movie_rating_dict = dict()
    u_predicted_rating = predict(U, V, user_id=uid)
    for movie_idx, pred_rating in enumerate(u_predicted_rating):
        movie_rating_dict[movie_idx] = pred_rating
    
    uidx = uid_2_idx[uid]
    sorted_dict = sorted(movie_rating_dict.items(), key=operator.itemgetter(1), reverse=True)       # 전체 영화를 대상으로 예측 rating을 기준으로 나열
    already_seen_movie_idxs = np.nonzero(ori_rating_matrix[uidx])[0]                                # 유저가 평점을매긴(이미 본) 영화들의 idx

    print_cnt = 0
    for idx, pred_rating in sorted_dict:
        if print_cnt == top_N:
            break
        if idx not in already_seen_movie_idxs:
            print(str(movie_dict[midx_2_id[idx]])+"    "+str(pred_rating))
            print_cnt = print_cnt+1


In [147]:
U, V = train(mean_subtracted_ratings)

Initial RMSE: 2.7255522172249265
Epoch:     0, RMSE: 0.9779512854132124, Learning Rate:0.01
Epoch:     1, RMSE: 0.9529294290248149, Learning Rate:0.01


KeyboardInterrupt: ignored

In [148]:
predicted_ratings = predict(U, V)
predicted_ratings += avg_ratings.reshape([-1,1])
print(predicted_ratings)

[[4.49875605 4.25876359 4.51960543 ... 4.39401113 3.89768776 4.20862012]
 [3.97291939 4.29052901 4.04835139 ... 4.00964466 3.37209102 3.9083888 ]
 [1.88284468 3.14692895 2.08277607 ... 2.61182789 2.62868073 2.38284943]
 ...
 [3.57481517 3.7114272  3.5701179  ... 4.10170264 2.71513602 4.31123356]
 [3.11639933 3.35366026 3.26504478 ... 2.56291284 2.829947   3.6245471 ]
 [4.0761744  4.05374665 4.3810409  ... 3.54262992 3.12751287 2.65956761]]


In [149]:
# 타겟 유저를 정해서, 해당 유저가 본 영화들의 분포를 뽑아내보기
# 유저의 추첨 결과가 올바르게 나왔는지 해석하기 위하여 해당 함수를 만들어서 추천 결과와 출력합니다.  
def print_user_preference(ori_rating_matrix, u_idx, type):
    '''
        type : ['avg', 'sum', 'cnt']
        avg 는 평균 평점, sum은 평점의 합, cnt은 평점 매긴 횟수
    '''
    target_hist = ori_rating_matrix[u_idx]
    target_seen_movie_idx =  np.nonzero(target_hist)[0]
    target_genre_dict = dict()
    target_genre_sum_rating_dict = dict()
    for movie_idx in target_seen_movie_idx:
        _, genre = movie_dict[midx_2_id[movie_idx]]
        for g in genre.split("|"):
            if g not in target_genre_dict.keys():
                target_genre_dict[g]=0
                target_genre_sum_rating_dict[g]=0
            target_genre_dict[g]=target_genre_dict[g]+1
            target_genre_sum_rating_dict[g] = target_genre_sum_rating_dict[g] + ori_rating_matrix[u_idx][movie_idx]

    # # 많이 본 횟수 대로 
    if type=='cnt':
        target_genre_dict = sorted(target_genre_dict.items(), key=operator.itemgetter(1), reverse=True)
        for k,v in target_genre_dict:
            print(k,v)
    
    # # 영화 평점의 합대로
    elif type=='sum':
        target_genre_sum_rating_dict = sorted(target_genre_sum_rating_dict.items(), key=operator.itemgetter(1), reverse=True)
        for k,v in target_genre_sum_rating_dict:
            print(k,v)
        
    # 영화 평균 평점 순서대로
    elif type=="avg":    
        target_genre_avg_rating_dict = dict()
        for k,v in target_genre_sum_rating_dict.items():
            target_genre_avg_rating_dict[k] = v/target_genre_dict[k]
        target_genre_avg_rating_dict = sorted(target_genre_avg_rating_dict.items(), key=operator.itemgetter(1), reverse=True)
        for k,v in target_genre_avg_rating_dict:
            print(k,v)
    
    else:
        target_genre_avg_rating_dict = dict()
        for k,v in target_genre_sum_rating_dict.items():
            target_genre_avg_rating_dict[k] = v/target_genre_dict[k]
        target_genre_avg_rating_dict = sorted(target_genre_avg_rating_dict.items(), key=operator.itemgetter(1), reverse=True)
        for k,v in target_genre_avg_rating_dict:
            print(k,v,target_genre_dict[k],target_genre_sum_rating_dict[k])

In [150]:
##### Check the RMSE of the test set ####### 
pred = predicted_ratings[test_ratings.nonzero()].flatten()
actual = test_ratings[test_ratings.nonzero()].flatten()

print("### Test RMSE ###")
print(np.sqrt(mean_squared_error(pred,actual)))

### Test RMSE ###
0.7625993219454933


In [151]:
musical_fan = 800
docu_fan = 850
horror_fan = 900
animation_fan =  950

In [152]:
# 해당 유저가 본 영화의 분포를 알고 싶으면 print_user_preference 함수를 실행시켜보세요!
# print_user_preference(rating_matrix, musical_fan, 'sum')
recommend_for_uid(musical_fan, rating_matrix, U, V, top_N=30)

('Wallace & Gromit: The Best of Aardman Animation (1996)', 'Adventure|Animation|Comedy')    2.3294221938963253
('Night on Earth (1991)', 'Comedy|Drama')    2.1299870079617826
('Fifty Shades Darker (2017)', 'Drama|Romance')    2.0203418228650447
('Annie (1982)', 'Children|Musical')    1.9505318797404674
('Ice Harvest, The (2005)', 'Action|Comedy|Crime|Thriller')    1.9433975110514679
('Party Girl (1995)', 'Comedy')    1.8879813799505716
('Broken English (1996)', 'Drama')    1.7824617153258704
('Lady Bird (2017)', 'Comedy')    1.7724658819755716
('Friday the 13th Part VIII: Jason Takes Manhattan (1989)', 'Horror')    1.755614196733208
('Last Night (1998)', 'Drama|Sci-Fi')    1.7509274420834746
('Harmonists, The (1997)', 'Drama')    1.7351091226040836
('Stoned (2005)', 'Drama')    1.730696143807202
('Bottle Rocket (1996)', 'Adventure|Comedy|Crime|Romance')    1.7278949295815687
('The Jinx: The Life and Deaths of Robert Durst (2015)', 'Documentary')    1.7048364068467667
('NeverEnding Stor

In [153]:
# 해당 유저가 본 영화의 분포를 알고 싶으면 print_user_preference 함수를 실행시켜보세요!
# print_user_preference(rating_matrix, docu_fan,'sum')
recommend_for_uid(docu_fan, rating_matrix, U, V, top_N=30)

('Annie (1982)', 'Children|Musical')    2.428396435676265
('Fifty Shades Darker (2017)', 'Drama|Romance')    2.1014870683723634
('Broken English (1996)', 'Drama')    1.9857706556868644
('NeverEnding Story III, The (1994)', 'Adventure|Children|Fantasy')    1.9343324391143413
('Annie Get Your Gun (1950)', 'Comedy|Musical|Romance|Western')    1.9155414911512876
('Untamed Heart (1993)', 'Drama|Romance')    1.8364445944914811
('Grease 2 (1982)', 'Comedy|Musical|Romance')    1.8145219955351415
('Pecker (1998)', 'Comedy|Drama')    1.7814486963096348
('Thousand Acres, A (1997)', 'Drama')    1.7740053400611009
('Jaws 2 (1978)', 'Horror|Thriller')    1.7620380874572448
('Day at the Races, A (1937)', 'Comedy|Musical')    1.7424369368883166
('Bridge, The (2006)', 'Documentary')    1.7307631618997217
('Jiro Dreams of Sushi (2011)', 'Documentary')    1.716750847210569
("Who's That Knocking at My Door? (1967)", 'Drama')    1.6791849277106157
('All of Me (1984)', 'Comedy|Fantasy')    1.672084269409141

In [154]:
# 해당 유저가 본 영화의 분포를 알고 싶으면 print_user_preference 함수를 실행시켜보세요!
# print_user_preference(rating_matrix, horror_fan,'sum')
recommend_for_uid(horror_fan, rating_matrix, U, V, top_N=30)

('Cold Comes the Night (2013)', 'Crime|Drama|Thriller')    2.070432850710189
('Rocky V (1990)', 'Action|Drama')    1.9837074514134583
('Spice World (1997)', 'Comedy')    1.970624012192709
('Forces of Nature (1999)', 'Comedy|Romance')    1.9530698758471616
('George Washington (2000)', 'Drama')    1.9128076100211377
('House of Wax (2005)', 'Horror|Thriller')    1.7725467375044932
('Memoirs of an Invisible Man (1992)', 'Comedy|Romance|Sci-Fi|Thriller')    1.7693879713497893
('Omega Code, The (1999)', 'Action')    1.767972128844468
('Last Knights (2015)', 'Action|Adventure')    1.7400057168845184
('House on Haunted Hill (1999)', 'Horror|Thriller')    1.7395817631946855
('Twin Dragons (Shuang long hui) (1992)', 'Action|Comedy')    1.7362606122792548
('Wrongfully Accused (1998)', 'Action|Comedy')    1.7120285814866738
('Man Apart, A (2003)', 'Action|Crime|Drama|Thriller')    1.7046081084555915
('Dobermann (1997)', 'Action|Crime')    1.6964988854553917
('Albatross (2011)', 'Drama')    1.69430

In [155]:
# 해당 유저가 본 영화의 분포를 알고 싶으면 print_user_preference 함수를 실행시켜보세요!
# print_user_preference(rating_matrix, horror_fan,'sum')
recommend_for_uid(animation_fan, rating_matrix, U, V, top_N=30)

('Louis C.K.: Shameless (2007)', 'Comedy')    1.3814449465930794
('Killing Fields, The (1984)', 'Drama|War')    1.3441525875273044
('The Artist (2011)', 'Comedy|Drama|Romance')    1.3429217913680478
("Guess Who's Coming to Dinner (1967)", 'Drama')    1.3370388670241222
('12 Angry Men (1957)', 'Drama')    1.3264739589002876
('Gentlemen of Fortune (Dzhentlmeny udachi) (1972)', 'Comedy|Crime|Drama|Mystery')    1.3207753272899934
('Baraka (1992)', 'Documentary')    1.3149367838020252
('Dragon Ball Z: Bojack Unbound (Doragon bôru Z 9: Ginga girigiri!! Butchigiri no sugoi yatsu) (1993)', 'Action|Adventure|Animation|Fantasy')    1.3005803577538022
('Yojimbo (1961)', 'Action|Adventure')    1.2870636198543985
("Devil's Playground (2002)", 'Documentary')    1.2813235644377237
('A Detective Story (2003)', 'Animation|Sci-Fi')    1.2797290089880549
('Black Mirror: White Christmas (2014)', 'Drama|Horror|Mystery|Sci-Fi|Thriller')    1.2784456359449394
("Man Bites Dog (C'est arrivé près de chez vous) 

# Test Code

In [157]:
################ Generating Synthetic Data #######################
synthetic_rating = np.zeros((5,10))

for i in range(10):
    random.seed(i)
    u_idx = random.randint(0,4)
    i_idx = random.randint(0,9)
    r_ui = random.randint(1,5)
    synthetic_rating[u_idx ,i_idx] = r_ui

synthetic_R = coo_matrix(synthetic_rating)
synthetic_R_zipped = list(zip(synthetic_R.row, synthetic_R.col, synthetic_R.data))
np.random.seed(7)
synthetic_U = np.random.rand(5, 3)
synthetic_V = np.random.rand(3, 10)

In [158]:
answer_U = np.load('./drive/MyDrive/data/others/answer_U.npy')
answer_V = np.load('./drive/MyDrive/data/others/answer_V.npy')
answer_rmse = np.load('./drive/MyDrive/data/others/answer_rmse.npy')

In [159]:
def test_code(sgd, rmse):
    U, V = sgd(synthetic_U, synthetic_V, synthetic_R_zipped, lr=0.01, lambda_u=0.1, lambda_v=0.1) 
    mse_u = mean_squared_error(U, answer_U)
    mse_v = mean_squared_error(V, answer_V)
    assert(mse_u <1e-2 or mse_v < 1e-2), 'calculated U, V is different with the answer : SGD 함수 오류'   ## 정답과 오차범위 내의 값이 아니면 assertion 오류

    rmse = calculate_rmse(synthetic_R, U, V, lambda_u=0.1, lambda_v=0.1)
    assert(abs(answer_rmse-rmse)<1e-05), 'calculated rmse is diferent with the answer : RMSE 함수 오류'   ## 정답과 오차범위 내의 값이 아니면 assertion 오류
    
    print("모든 함수 알맞게 구현됨")


In [160]:
test_code(SGD, calculate_rmse)

모든 함수 알맞게 구현됨
